In [1]:
!pip install python-dotenv
!pip install snowflake-connector-python

In [2]:
from dotenv import load_dotenv
import os
import pyspark
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import time

In [3]:
# Cargo mis variables de entorno
load_dotenv()

# Genero el dict de datos para conectarme con Snowflake
credencialesSnowflakeRaw = {
    "sfURL" : os.getenv("SNOWFLAKE_URL"),
    "sfUser" :  os.getenv("SNOWFLAKE_USER"),
    "sfPassword" : os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase" : os.getenv("SNOWFLAKE_DATABASE"),
    "sfSchema" : os.getenv("SNOWFLAKE_SCHEMA_RAW"),
    "sfWarehouse" : os.getenv("SNOWFLAKE_WAREHOUSE"),
    "sfRole" : os.getenv("SNOWFLAKE_ROLE"),
}

print(f"Estas son mis credenciales para Snowflake: {credencialesSnowflakeRaw}")

# Creo SparkSession para conexión con Snowflake
spark = (SparkSession.builder.appName("IngestaNewYorkTaxis").config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.30,net.snowflake:spark-snowflake_2.12:2.9.0-spark_3.1").getOrCreate())

print(spark)
print("Spark Version : " + spark.version)

# Ejecuto una query de prueba para validar comunicacion con Snowflake
query = "SELECT current_version()"

df = spark.read.format("snowflake").options(**credencialesSnowflakeRaw).option("query", query).load()

df.show()

Estas son mis credenciales para Snowflake: {'sfURL': 'LSNDJXB-RHC82043.snowflakecomputing.com', 'sfUser': 'usuario_spark', 'sfPassword': 'EstudianteEstudiante64', 'sfDatabase': 'NY_TAXI', 'sfSchema': 'RAW', 'sfWarehouse': 'WAREHOUSE_TAXIS', 'sfRole': 'rol_pocos_privilegios'}
Spark Version : 3.5.0
+-------------------+
|"CURRENT_VERSION()"|
+-------------------+
|             9.32.1|
+-------------------+



In [4]:
def ingestar_zones_a_raw():
    SOURCE_PATH = os.getenv("SOURCE_PATH")
    path_url = f"{SOURCE_PATH}/misc/taxi_zone_lookup.csv"
    local_path = f"/tmp/taxiZones.parquet"
    
    # Descargo el archivo en carpeta temporal para posteriormente leerlo
    try:
        r = requests.get(path_url, stream=True)
        if r.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=10000000):
                    f.write(chunk)
        else:
            print(f"Archivo no encontrado en {path_url} (status {r.status_code})")
            return None
    except Exception as e:
        print(f"Error descargando {path_url}: {e}")
        return None
    else:
        print(f"Archivo obtenido exitosamente de: {path_url}")
    
    # Leo el archivo en un df de Spark
    try:
        df = spark.read.csv(local_path, header="true")
    except Exception as e:
        print(f"No se pudo leer {local_path}: {e}")
        return None
    else:
        print(f"Archivo leido exitosamente por Spark: {local_path}")

    conteoFilas = df.count()
    print(f"Ingestando hacia Snowflake datos de Zonas de Taxis. Total de filas: {conteoFilas}")

    try:
        df.write.format("snowflake") \
            .options(**credencialesSnowflakeRaw) \
            .option("dbtable", f"NY_TAXI_RAW_TAXI_ZONES") \
            .mode("overwrite") \
            .save()
    except Exception as e2:
        print(f"Error con tabla temporal: {e2}")
        return None
    else:
        print("Zonas de taxis exportadas correctamente a Raw de Snowflake")

    #Me aseguro de eliminar el archivo parquet temporal
    try:
        os.remove(local_path)
        print(f"Archivo parquet temporal removido: {local_path}")
    except OSError as e:
        print(f"No se pudo remover el archivo parquet temporal {local_path}: {e}")

    #Retorno datos para tabla de conteos de datos consumidos por run
    return {
        "count": conteoFilas,
    }

In [5]:
#Cargo Zonas de Taxis en Snowflake
zonasTaxisIngesta=ingestar_zones_a_raw()
print(zonasTaxisIngesta)

Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Archivo leido exitosamente por Spark: /tmp/taxiZones.parquet
Ingestando hacia Snowflake datos de Zonas de Taxis. Total de filas: 265
Zonas de taxis exportadas correctamente a Raw de Snowflake
Archivo parquet temporal removido: /tmp/taxiZones.parquet
{'count': 265}


In [11]:
#Defino funciones tipicas de checkpoint para en caso de fallo no ingestar datos desde cero

def save_checkpoint(year, month, CHECKPOINT_FILE_COMBINADO):
    with open(CHECKPOINT_FILE_COMBINADO, "w") as f:
        json.dump({"year": year, "month": month}, f)

def load_checkpoint(CHECKPOINT_FILE_COMBINADO):
    if os.path.exists(CHECKPOINT_FILE_COMBINADO):
        with open(CHECKPOINT_FILE_COMBINADO, "r") as f:
            return json.load(f)
    return {"year": 0, "month": 0}

In [12]:
# Función para aplicar normalización
def normalizar_catalogos(df):

    df_normalizado = df

    # Defino mapeos para normalización. Los defino como base en los data dictionary que hay en la pagina de los datos
    mapeo_payment_type = {
        0: 'Flex Fare trip',
        1: 'Credit card',
        2: 'Cash',
        3: 'No charge',
        4: 'Dispute',
        5: 'Unknown',
        6: 'Voided trip'
    }
    
    mapeo_rate_code = {
        1: 'Standard rate',
        2: 'JFK',
        3: 'Newark',
        4: 'Nassau or Westchester',
        5: 'Negotiated fare',
        6: 'Group ride',
        99: 'Unknown'
    }
    
    mapeo_vendor = {
        1: 'Creative Mobile Technologies,  LLC',
        2: 'Curb Mobility, LLC',
        6: 'Myle Technologies Inc',
        7: 'Helix'
    }
        
    # Normalizo payment_type
    if 'PAYMENT_TYPE' in df.columns:
        df_normalizado = df_normalizado.withColumn('PAYMENT_TYPE_NORMALIZADO',F.coalesce(
        F.create_map([F.lit(x) for item in mapeo_payment_type.items() for x in item])[F.col('PAYMENT_TYPE').cast('int')],
        F.lit('Unknown')))
        
    # Normalizo RateCode
    if 'RATECODEID' in df.columns:
        df_normalizado = df_normalizado.withColumn('RATE_CODE_NORMALIZADO',F.coalesce(
        F.create_map([F.lit(x) for item in mapeo_rate_code.items() for x in item])[F.col('RATECODEID').cast('int')],
        F.lit('Unknown')))
      
    # Normalizo VendorID
    if 'VENDORID' in df.columns:
        df_normalizado = df_normalizado.withColumn('VENDOR_NORMALIZADO',F.coalesce(
        F.create_map([F.lit(x) for item in mapeo_vendor.items() for x in item])[F.col('VENDORID').cast('int')],
        F.lit('Unknown')))
        
    return df_normalizado

In [13]:
def guardar_con_reintentos(df, tabla, reintentos=3):
    
    for intento in range(reintentos):
        try:
            print(f"Intento {intento + 1} de {reintentos} para guardar en {tabla}")
            df.write.format("snowflake") \
                .options(**credencialesSnowflakeRaw) \
                .option("dbtable", tabla) \
                .mode("append") \
                .save()
            print(f"Tabla {tabla} actualizada exitosamente")
            return True
        except Exception as e:
            print(f"Error en intento {intento + 1}: {str(e)[:200]}...")
            if intento < reintentos - 1:
                wait_time = 30 
                print(f"Esperando {wait_time} segundos antes del reintento")
                time.sleep(wait_time)
            else:
                print(f"Todos los intentos fallaron para {tabla}")
                raise e
    return False

def guardar_por_lotes(df, tabla, batch_size=100000):
    
    total_count = df.count()
    num_partitions = max(1, total_count // batch_size)
    
    print(f"Dividiendo {total_count} filas en {num_partitions} particiones")
    
    df_reparticionado = df.repartition(num_partitions)
    
    return guardar_con_reintentos(df_reparticionado, tabla)


In [16]:
def normalizar_y_unificar_taxis():

    #Unifico tablas yellow y green, normalizando catálogos de payment_type, rate_code y vendor
    print("Iniciando proceso de la unificacion y el enriquecimiento de datos de viajes de taxis en NY")
    # Leo las tablas desde Snowflake
    try:
        df_yellow = spark.read.format("snowflake") \
            .options(**credencialesSnowflakeRaw) \
            .option("dbtable", "NY_TAXI_RAW_YELLOW") \
            .load()
        
        df_green = spark.read.format("snowflake") \
            .options(**credencialesSnowflakeRaw) \
            .option("dbtable", "NY_TAXI_RAW_GREEN") \
            .load()
            
        print("Tablas yellow y green leídas exitosamente desde Snowflake")
        
    except Exception as e:
        print(f"Error leyendo tablas desde Snowflake: {e}")
        return None

    # Normalizo ambas tablas
    try:
        print("Normalizando tabla yellow")
        df_yellow_normalizado = normalizar_catalogos(df_yellow)
        
        print("Normalizando tabla green")
        df_green_normalizado = normalizar_catalogos(df_green)
        
        print("Catálogos normalizados exitosamente")
        
    except Exception as e:
        print(f"Error en normalización de catálogos: {e}")
        return None

    # Unifico las tablas
    try:
        
        columnas_yellow = set(df_yellow_normalizado.columns)
        columnas_green = set(df_green_normalizado.columns)
        CHECKPOINT_FILE_COMBINADO="checkpointCargaUnificado.json"
        
        if ( columnas_yellow == columnas_green):
            df_unificado = df_yellow_normalizado.select(*sorted(columnas_yellow)).unionByName(df_green_normalizado.select(*sorted(columnas_yellow)))
            conteo_filas = df_unificado.count()
            print(f"Tablas unificadas exitosamente. Total de filas: {conteo_filas}")

            lista_years = sorted([int(item) for item in (os.getenv("YEARS").split(','))])
            lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
            
            checkpoint=load_checkpoint(CHECKPOINT_FILE_COMBINADO)
            
            print(f"checkpoint: {checkpoint}")

            if ( checkpoint != {"year": 0, "month": 0} and (int(checkpoint["month"]) in lista_months) and (int(checkpoint["year"]) in lista_years)):    
                if ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ):
                    lista_years= lista_years[lista_years.index(checkpoint["year"])+1:]
                elif ( int(checkpoint["month"]) != lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ): 
                    lista_years= lista_years[lista_years.index(checkpoint["year"]):]
                    lista_months= lista_months[lista_months.index(checkpoint["month"])+1:]
    
            for year in lista_years:
                for month in lista_months:
                    df_lote = df_unificado.filter((F.col("SOURCE_YEAR") == int(year)) & (F.col("SOURCE_MONTH") == int(month)))
                    print(f"Se han filtrado datos de year {year} + month {month}. Se procedera a cargarlos en la base de Snowflake")
                    guardar_por_lotes(df_lote, "NY_TAXI_TRIPS_UNIFIED", batch_size=100000)
                    print(f"Guardados correctamente datos de taxis de year {year} + month {month}")
                    save_checkpoint(year, month, CHECKPOINT_FILE_COMBINADO) 
                lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
                
        else:
            print(f"No se pudo unificar tablas")
            return None
        
    except Exception as e:
        print(f"Error unificando tablas: {e}")
        return None
    
    # Retorno estadísticas
    return {"total_filas": conteo_filas,"filas_yellow": df_yellow_normalizado.count(),"filas_green": df_green_normalizado.count()}

In [17]:
resultado = normalizar_y_unificar_taxis()

Iniciando proceso de la unificacion y el enriquecimiento de datos de viajes de taxis en NY
Tablas yellow y green leídas exitosamente desde Snowflake
Normalizando tabla yellow
Normalizando tabla green
Catálogos normalizados exitosamente
Tablas unificadas exitosamente. Total de filas: 852432668
checkpoint: {'year': 0, 'month': 0}
Se han filtrado datos de year 2015 + month 1. Se procedera a cargarlos en la base de Snowflake
Dividiendo 14249528 filas en 142 particiones
Intento 1 de 3 para guardar en NY_TAXI_TRIPS_UNIFIED
Tabla NY_TAXI_TRIPS_UNIFIED actualizada exitosamente
Guardados correctamente datos de taxis de year 2015 + month 1
Se han filtrado datos de year 2015 + month 2. Se procedera a cargarlos en la base de Snowflake
Dividiendo 14017224 filas en 140 particiones
Intento 1 de 3 para guardar en NY_TAXI_TRIPS_UNIFIED
Tabla NY_TAXI_TRIPS_UNIFIED actualizada exitosamente
Guardados correctamente datos de taxis de year 2015 + month 2
Se han filtrado datos de year 2015 + month 3. Se proce